In [1]:
import json
import pandas as pd
from openai import OpenAI
from tqdm.auto import tqdm

In [2]:
with open('documents-with-ids.json', 'rt',  encoding='utf-8') as f_in:
    documents = json.load(f_in)

documents[20]


{'text': 'El Centro Histórico tiene un 34.9% de viviendas con apenas un dormitorio, indicando una condición de hacinamiento.',
 'section': 'Bienestar Social',
 'question': '¿Cuál es el porcentaje de viviendas con apenas un dormitorio en el Centro Histórico?',
 'topic': 'PMD',
 'id': '9079cdc6'}

In [3]:
df_ground_truth = pd.read_csv('ground-truth-data.csv')
ground_truth = df_ground_truth.to_dict(orient='records')
ground_truth[20]

{'question': '¿Cuál es la relación entre turismo y economía local?',
 'topic': 'PMD',
 'document': '21b7970e'}

In [4]:
doc_idx = {d['id']: d for d in documents}
doc_idx['21b7970e']['text']

'El turismo representa una actividad de primera importancia para el desarrollo económico de Puebla.'

In [5]:
df_llama = pd.read_csv('results-llama.csv')

In [6]:
df_llama.head()

,answer_llm,answer_orig,document,question,topic
0,La acción que ayudará al Eje 4 a proteger los ...,El Eje 4 contribuirá al repoblamiento y mejora...,335b2f94,¿Qué acción ayudará al Eje 4 a proteger los el...,PMD
1,La respuesta a la pregunta es:\n\nEl Eje 4 con...,El Eje 4 contribuirá al repoblamiento y mejora...,335b2f94,En qué medida el Eje 4 contribuirá al mejorami...,PMD
2,"Basándome en el contexto proporcionado, los ob...",El Plan de Manejo del Centro Histórico de Pueb...,805017f9,¿Cuáles son los objetivos principales del Plan...,PMD
3,"Basándome en el contexto proporcionado, las ac...",El Plan de Manejo del Centro Histórico de Pueb...,805017f9,¿Qué acciones se llevarán a cabo para lograr u...,PMD
4,La política abordada por los programas parcial...,Los programas parciales de desarrollo urbano d...,017273c8,¿Qué política abordan los programas parciales ...,PMD


In [7]:
client = OpenAI(
    base_url='http://localhost:11434/v1/',
    api_key='ollama',
)

def llm(prompt, model='llama3.1'):
    response = client.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [8]:
prompt1_template = """
Eres un evaluador experto para un sistema de Generación Aumentada por Recuperación (RAG).
Tu tarea es analizar la relevancia de la respuesta generada en comparación con la respuesta original proporcionada.
Con base en la relevancia y similitud de la respuesta generada con la respuesta original, la clasificarás
como "NO_RELEVANTE", "PARCIALMENTE_RELEVANTE" o "RELEVANTE".

Aquí están los datos para la evaluación:

Respuesta Original: {answer_orig}
Pregunta Generada: {question}
Respuesta Generada: {answer_llm}

Por favor, analiza el contenido y contexto de la respuesta generada en relación con la respuesta original
y proporciona tu evaluación en JSON sin usar bloques de código:

{{
  "Relevancia": "NO_RELEVANTE" | "PARCIALMENTE_RELEVANTE" | "RELEVANTE",
  "Explicación": "[Proporciona una breve explicación para tu evaluación]"
}}
""".strip()


In [9]:
prompt2_template = """
Eres un evaluador experto para un sistema de Generación Aumentada por Recuperación (RAG).
Tu tarea es analizar la relevancia de la respuesta generada en relación con la pregunta dada.
Con base en la relevancia de la respuesta generada, la clasificarás
como "NO_RELEVANTE", "PARCIALMENTE_RELEVANTE" o "RELEVANTE".

Aquí están los datos para la evaluación:

Pregunta: {question}
Respuesta Generada: {answer_llm}

Por favor, analiza el contenido y contexto de la respuesta generada en relación con la pregunta
y proporciona tu evaluación en JSON sin usar bloques de código:

{{
  "Relevancia": "NO_RELEVANTE" | "PARCIALMENTE_RELEVANTE" | "RELEVANTE",
  "Explicación": "[Proporciona una breve explicación para tu evaluación]"
}}
""".strip()


In [10]:
df_sample = df_llama.sample(n=50, random_state=1)

In [11]:
df_sample.head()

,answer_llm,answer_orig,document,question,topic
207,La respuesta a tu pregunta es:\n\nEl municipio...,El Centro Histórico del Municipio de Puebla fu...,7f5bb88b,¿Cuándo fue reconocido oficialmente como Zona ...,PPDUS
82,La red urbana de transporte articulado (RUTA) ...,El Municipio de Puebla cuenta con una vasta re...,cfeffd05,¿Qué tipo de transporte público cubre la zona ...,PMD
185,Según la información proporcionada en el conte...,La construcción de redes de alcantarillado deb...,884ed4cb,¿Qué elementos deben garantizarse siempre en l...,PPDUS
367,"No, las unidades de servicio exclusiva de la L...",La Línea 2 de RUTA tiene unidades de servicio ...,af455f9a,¿Incluyen las unidades de servicio exclusivo d...,RUTA
65,"La respuesta a la pregunta es:\n\n""Se necesita...",Se necesitan instrumentos específicos para la ...,3c367131,¿Cuál es el desarrollo urbano que requiere ins...,PMD


In [12]:
samples = df_sample.to_dict(orient='records')

In [13]:
record = samples[0]
record

{'answer_llm': 'La respuesta a tu pregunta es:\n\nEl municipio de Puebla fue reconocido oficialmente como Zona de Monumentos Históricos en 1977, mediante el Decreto que lleva la fecha del 18 de Noviembre de dicho año.',
 'answer_orig': 'El Centro Histórico del Municipio de Puebla fue decretado como Zona de Monumentos Históricos en 1977, sin embargo su límite original fue generado a partir una estructura urbana que se ha modificado en algunas zonas con el paso de los años.',
 'document': '7f5bb88b',
 'question': '¿Cuándo fue reconocido oficialmente como Zona de Monumentos Históricos?',
 'topic': 'PPDUS'}

In [14]:
prompt = prompt1_template.format(**record)
print(prompt)

Eres un evaluador experto para un sistema de Generación Aumentada por Recuperación (RAG).
Tu tarea es analizar la relevancia de la respuesta generada en comparación con la respuesta original proporcionada.
Con base en la relevancia y similitud de la respuesta generada con la respuesta original, la clasificarás
como "NO_RELEVANTE", "PARCIALMENTE_RELEVANTE" o "RELEVANTE".

Aquí están los datos para la evaluación:

Respuesta Original: El Centro Histórico del Municipio de Puebla fue decretado como Zona de Monumentos Históricos en 1977, sin embargo su límite original fue generado a partir una estructura urbana que se ha modificado en algunas zonas con el paso de los años.
Pregunta Generada: ¿Cuándo fue reconocido oficialmente como Zona de Monumentos Históricos?
Respuesta Generada: La respuesta a tu pregunta es:

El municipio de Puebla fue reconocido oficialmente como Zona de Monumentos Históricos en 1977, mediante el Decreto que lleva la fecha del 18 de Noviembre de dicho año.

Por favor, a

In [15]:
answer = llm(prompt)

In [16]:
answer

'{\n\n"Relevancia": "PARTICAMENTE_RELEVANTE",\n\n"Explanation": "Si bien la respuesta generated proporciona información correcta sobre la fecha en que se reconoció oficialmente al municipio de Puebla como Zona de Monumentos Históricos, el contenido no aborda específicamente el cambio en los límites originales del Centro Histórico debido a las modificaciones urbanas con el paso de los años. La respuesta original es más amplia y contextualiza la designación como Zona de Monumentos Históricos dentro del marco de las características histórico-urbanísticas del área, mientras que la respuesta generada solo proporciona la fecha y se centra en el reconocimiento oficial sin referirse explícitamente a estas modificaciones."\n\n}'

# Prompt 1

In [17]:
evaluations = []

for record in tqdm(samples):
    prompt = prompt1_template.format(**record)
    evaluation = llm(prompt)
    evaluations.append(evaluation)

  0%|          | 0/50 [00:00<?, ?it/s]

In [258]:
json_evaluations = []
failed_parses = []

       
for i, str_eval in enumerate(evaluations):
    try:
        json_eval = json.loads(str_eval)
        json_evaluations.append(json_eval)
    except json.JSONDecodeError:
        failed_parses.append(i)

In [259]:
len(json_evaluations)

50

In [260]:
failed_parses

[]

In [262]:
df_evaluations = pd.DataFrame(json_evaluations)

In [283]:
df_evaluations[df_evaluations.Relevancia == 'NO_RELEVANTE'] #.to_dict(orient='records')

,Relevancia,Explicación
3,NO_RELEVANTE,La respuesta generada es incorrecta y difiere ...
5,NO_RELEVANTE,La respuesta generada no menciona específicame...
13,NO_RELEVANTE,La respuesta generada no se relaciona con el c...
17,NO_RELEVANTE,La respuesta generada no aborda la cuestión de...
18,NO_RELEVANTE,La respuesta generada no es relevante con la p...
20,NO_RELEVANTE,La respuesta generada no coincide con el conte...
21,NO_RELEVANTE,La respuesta generada no contiene la informaci...
26,NO_RELEVANTE,La respuesta generada no proporciona informaci...
29,NO_RELEVANTE,La respuesta generada no proporciona informaci...
33,NO_RELEVANTE,La respuesta generada cambia el contexto de la...


In [288]:
samples[3]

{'answer_llm': 'No, las unidades de servicio exclusiva de la Línea 2 de RUTA no incluyen padrones entre sus características. Según la información proporcionada, dichas unidades poseen 2 zafiros y 2 articulados con características específicas para atender el transporte de mujeres y menores de edad en el municipio de Puebla.',
 'answer_orig': 'La Línea 2 de RUTA tiene unidades de servicio exclusivo para mujeres y menores de edad que incluyen 2 padrones y 2 articulados.',
 'document': 'af455f9a',
 'question': '¿Incluyen las unidades de servicio exclusivo de la Línea 2 de RUTA algunos padrones entre sus características?',
 'topic': 'RUTA'}

In [284]:
df_evaluations.Relevancia.value_counts()

Relevancia
RELEVANTE                 26
NO_RELEVANTE              13
PARCIALMENTE_RELEVANTE    11
Name: count, dtype: int64

In [287]:
df_evaluations.iloc[42, 0] 

'PARCIALMENTE_RELEVANTE'

In [293]:
df_evaluations.to_csv('evaluations_aqa.csv', index=False)

# Prompt 2

In [289]:
prompt = prompt2_template.format(**record)
print(prompt)

Eres un evaluador experto para un sistema de Generación Aumentada por Recuperación (RAG).
Tu tarea es analizar la relevancia de la respuesta generada en relación con la pregunta dada.
Con base en la relevancia de la respuesta generada, la clasificarás
como "NO_RELEVANTE", "PARCIALMENTE_RELEVANTE" o "RELEVANTE".

Aquí están los datos para la evaluación:

Pregunta: ¿Cuántas acciones se generarán para la conservación y preservación de inmuebles en los polígonos de actuación del Centro Histórico?
Respuesta Generada: Según la información proporcionada, dentro de los polígonos de actuación del Centro Histórico se planean generar 115 acciones de conservación y preservación en inmuebles.

Por favor, analiza el contenido y contexto de la respuesta generada en relación con la pregunta
y proporciona tu evaluación en JSON sin usar bloques de código:

{
  "Relevancia": "NO_RELEVANTE" | "PARCIALMENTE_RELEVANTE" | "RELEVANTE",
  "Explicación": "[Proporciona una breve explicación para tu evaluación]"


In [290]:
answer = llm(prompt)

In [291]:
answer

'{\n  "Relevancia": "RELEVANTE",\n  "Explicación": La respuesta generada proporciona información directa y específica sobre el número de acciones planeadas para la conservación y preservación en inmuebles dentro del área especificada, lo cual responde de manera precisas a la pregunta formulada.'

In [292]:
evaluations_2 = []

for record in tqdm(samples):
    prompt = prompt2_template.format(**record)
    evaluation = llm(prompt)
    evaluations_2.append(evaluation)

  0%|          | 0/50 [00:00<?, ?it/s]

In [377]:
json_evaluations_2 = []
failed_parses_2 = []

       
for i, str_eval in enumerate(evaluations_2):
    try:
        json_eval = json.loads(str_eval)
        json_evaluations_2.append(json_eval)
    except json.JSONDecodeError:
        failed_parses_2.append(i)

In [378]:
failed_parses_2

[]

In [400]:
evaluations_2[22]

'{"Relevancia": "RELEVANTE", \n "Explicación": "La respuesta generada tiene directa relación con la pregunta, proporcionando información específica sobre las acciones que se llevan a cabo en el área residencial del Centro Histórico para promover el uso de bicicletas y transporte público. La evidencia de estrategias específicas como la creación de ciclovías, mejorar el transporte público y facilitar rutas peatonales seguras demuestra una conexión clara a la pregunta."}'

In [380]:
df_evaluations_2 = pd.DataFrame(json_evaluations_2)

In [396]:
df_evaluations_2[df_evaluations_2.Relevancia == 'NO_RELEVANTE'] #.to_dict(orient='records')

,Relevancia,Explicación
2,NO_RELEVANTE,La respuesta generada menciona la hermeticidad...
3,NO_RELEVANTE,La respuesta no aborda directamente la inclusi...
5,NO_RELEVANTE,La respuesta generada no proporciona informaci...
6,NO_RELEVANTE,La respuesta proporcionada no aborda concretam...
14,NO_RELEVANTE,La respuesta generada no proporciona informaci...
18,NO_RELEVANTE,La respuesta generada no proporciona informaci...
21,NO_RELEVANTE,La respuesta proporcionada no aborda de manera...
24,NO_RELEVANTE,La respuesta generada no aborda la pregunta de...
26,NO_RELEVANTE,La respuesta generada es completamente irrelev...
27,NO_RELEVANTE,La respuesta generada se refiere específicamen...


In [395]:
df_evaluations_2.Relevancia.value_counts()

Relevancia
RELEVANTE                 19
NO_RELEVANTE              16
PARCIALMENTE_RELEVANTE    15
Name: count, dtype: int64

In [394]:
df_evaluations_2.iloc[27, 0] 

'NO_RELEVANTE'

In [398]:
df_evaluations_2.to_csv('evaluations_qa.csv', index=False)